# LangChain Agent Executor

In this chapter, we will continue from the [introduction to agents](https://aurelio.ai/learn/langchain-agents-intro) and dive deeper into agents. Learning how to build our custom agent execution loop for v0.3 of LangChain.

## What is the Agent Executor?

When we talk about agents, a significant part of an "agent" is simple code logic,
iteratively rerunning LLM calls and processing their output. The exact logic varies
significantly, but one well-known example is the **ReAct** agent.

![ReAct process](../assets/ai-agents-00.png)

**Re**ason + **Act**ion (ReAct) agents use iterative _reasoning_ and _action_ steps to
incorporate chain-of-thought and tool-use into their execution. During the _reasoning_
step, the LLM generates the steps to take to answer the query. Next, the LLM generates
the _action_ input, which our code logic parses into a tool call.

![Agentic graph of ReAct](../assets/ai-agents-01.png)

Following our action step, we get an observation from the tool call. Then, we feed the
observation back into the agent executor logic for a final answer or further reasoning
and action steps.

The agent and agent executor we will be building will follow this pattern.

## Step 1: Understanding the Big Picture 
### What is an Agent?
Think of an agent like a smart assistant that can:
- Understand what you want
- Decide which tools to use
- Use those tools step by step
- Give you a final answer

### How Does It Work?
```
You: "What is 5 + 3 multiplied by 2?"

Agent thinks: 
1. "I need to add 5 + 3 first"
2. "Then multiply the result by 2"
3. "Let me use my add tool, then multiply tool"

Agent: "The answer is 16"
```

### Why Build Our Own?
- **Free**: No API costs
- **Private**: Your data stays on your computer  
- **Customizable**: Add any tools you want
- **Educational**: Learn how AI agents really work

---

## Step 2: Setting Up Our Environment 🛠️

### Step 2a: Install Required Packages
```python
# First, let's install what we need
# Run this in your terminal or notebook:
# pip install langchain-community langchain-core ollama
```

### Step 2b: Import Basic Libraries

In [1]:


# Think of imports like getting tools from a toolbox
import json  # For handling structured data (like recipes)
import re   # For finding patterns in text (like finding phone numbers)
from typing import Dict, List, Any, Optional  # For being clear about data types

print("Basic libraries imported!")


Basic libraries imported!


**Why do we need these?**
- `json`: Our agent will "speak" in JSON format (structured data)
- `re`: To find and extract JSON from the model's responses
- `typing`: To make our code clear and prevent bugs


### Step 2c: Import LangChain Components

In [2]:
# LangChain is like a toolkit for building AI applications
from langchain_core.tools import tool  # For creating our math tools
from langchain_core.prompts import ChatPromptTemplate  # For creating instructions
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage  # For conversation
from langchain_community.llms import Ollama  # For connecting to our local AI model

print("LangChain components imported!")

LangChain components imported!



**What is LangChain?**
- It's like LEGO blocks for AI applications
- Provides pre-built components we can snap together
- Makes building AI apps much easier

---


## Step 3: Creating Simple Math Tools 

### Step 3a: Understanding the @tool Decorator


In [3]:
from langchain.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add two numbers together."""
    return x + y

# Call using invoke
result = add.invoke({"x": 5, "y": 3})
print(f"Tool result: {result}")

Tool result: 8.0


**What just happened?**
- The `@tool` decorator turns our function into something special
- It keeps track of the function name and description
- The description in triple quotes is important - it tells our AI what this tool does

### Step 3b: Creating Our Basic Math Tools

In [4]:
@tool
def subtract(x: float, y: float) -> float:
    """Subtract x from y (y - x)."""
    result = y - x
    print(f" Calculating {y} - {x} = {result}")
    return result

@tool  
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    result = x * y
    print(f" Multiplying {x} × {y} = {result}")
    return result

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise x to the power of y (x^y)."""
    result = x ** y
    print(f" Calculating {x}^{y} = {result}")
    return result

print(" All math tools created!")


 All math tools created!



**Why these specific tools?**
- They cover basic math operations
- Each has a clear, single purpose
- The print statements help us debug (see what's happening)


### Step 3c: Creating the Special "Final Answer" Tool


In [5]:
@tool
def final_answer(answer: str, tools_used: List[str]) -> Dict[str, Any]:
    """Provide the final answer to the user.
    
    Args:
        answer: The final answer in natural language
        tools_used: List of tool names that were used
    """
    result = {
        "answer": answer,
        "tools_used": tools_used
    }
    print(f"Final Answer: {answer}")
    print(f"Tools Used: {tools_used}")
    return result

print("Final answer tool created!")


Final answer tool created!


**Why do we need a "final answer" tool?**
- It signals when our agent is done calculating
- It provides a nice summary of what was done
- It gives us the answer in human-friendly language

### Step 3d: Organizing Our Tools

In [6]:
# Put all our tools in a list
tools = [add, subtract, multiply, exponentiate, final_answer]

# Create a dictionary to easily find tools by name
name2tool = {tool.name: tool.func for tool in tools}

print("Available tools:")
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}: {tool.description}")

print(f"\nTool dictionary created with {len(name2tool)} tools")

Available tools:
1. add: Add two numbers together.
2. subtract: Subtract x from y (y - x).
3. multiply: Multiply two numbers together.
4. exponentiate: Raise x to the power of y (x^y).
5. final_answer: Provide the final answer to the user.

    Args:
        answer: The final answer in natural language
        tools_used: List of tool names that were used

Tool dictionary created with 5 tools


**What's this dictionary for?**
- When our AI says "use the add tool", we need to find the actual function
- The dictionary lets us look up tools by name quickly
- Like a phone book, but for functions!


## Step 4: Understanding Why Ollama is Different 

### Step 4a: The OpenAI Way (What We Can't Do)

In [7]:
# This is what the original notebook did with OpenAI:
# 
# llm.bind_tools(tools, tool_choice="any")
# 
# This magically tells OpenAI about our tools and it can call them directly
# But Ollama doesn't have this feature!

print("Ollama doesn't support bind_tools()")
print("So we'll build our own system!")


Ollama doesn't support bind_tools()
So we'll build our own system!


### Step 4b: Our Creative Solution

### Instead of magic function calling, we'll:
- 1. Tell the AI about our tools in plain English
- 2. Ask it to respond with JSON format
- 3. Parse that JSON to see which tool it wants to use
- 4. Run the tool ourselves
- 5. Give the result back to the AI

## Step 5: Setting Up Ollama Connection 

### Step 5a: Install and Start Ollama

In [8]:
# Before running this code, you need to:
# 1. Install Ollama: https://ollama.ai
# 2. Download a model: ollama pull llama2
# 3. Start Ollama: ollama serve

print(" Ollama setup checklist:")
print(" Ollama installed")
print(" Model downloaded (ollama pull llama2)")  
print(" Ollama server running (ollama serve)")
print("\nIf any of these aren't done, do them first!")

 Ollama setup checklist:
 Ollama installed
 Model downloaded (ollama pull llama2)
 Ollama server running (ollama serve)

If any of these aren't done, do them first!


### Step 5b: Connect to Ollama

In [9]:
# Import necessary libraries
from langchain_core.tools import tool
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
import json
import requests
from datetime import datetime
from IPython.display import display, Markdown

# Initialize the Ollama LLM
llm = Ollama(
    model="llama3.2",  # Model name
    temperature=0.0,    # Lower temperature for more deterministic outputs
    base_url="http://localhost:11434"  # Default Ollama server URL
)

# Test the connection
response = llm.invoke("Hello, are you working?")
print(response)

C:\Users\USERAS\AppData\Local\Temp\ipykernel_9500\3214559778.py:13: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


I'm here and ready to help. How can I assist you today?


In [10]:

# Test the connection
try:
    test_response = llm.invoke("Hello! Can you count to 3?")
    print(f" Ollama connected! Test response: {test_response}")
except Exception as e:
    print(f" Connection failed: {e}")
    print("Make sure Ollama is running with 'ollama serve'")


 Ollama connected! Test response: 1, 2, 3.


**Temperature Explained:**
- 0.0 = Robot-like, same answer every time
- 0.5 = Balanced  
- 1.0 = Very creative, different answers each time
- For math, we want consistency, so we use 0.0

---

## Step 6: Creating Tool Descriptions for Our AI

### Step 6a: Understanding the Challenge

In [11]:
# Our AI needs to know what tools are available
# Since we can't use bind_tools(), we'll describe them in text

print("The Challenge:")
print("How do we tell our AI about available tools?")
print("Answer: We'll write descriptions it can understand!")

The Challenge:
How do we tell our AI about available tools?
Answer: We'll write descriptions it can understand!


### Step 6b: Creating Tool Schema

In [12]:
def create_tool_schema() -> str:
    """Create a human-readable description of our tools."""
    
    descriptions = []
    
    for tool in tools:
        # Get information about each tool
        tool_info = f"""
                    Tool: {tool.name}
                    Description: {tool.description}
                    Use when: {_get_usage_hint(tool.name)}
                    """
        descriptions.append(tool_info)
    
    return "\n".join(descriptions)

def _get_usage_hint(tool_name: str) -> str:
    """Provide hints about when to use each tool."""
    hints = {
        "add": "you need to add two numbers",
        "subtract": "you need to subtract one number from another",
        "multiply": "you need to multiply two numbers", 
        "exponentiate": "you need to raise a number to a power",
        "final_answer": "you're ready to give the final answer to the user"
    }
    return hints.get(tool_name, "appropriate for this tool")

# Create our tool schema
tool_schema = create_tool_schema()
print("Tool Schema Created:")
print(tool_schema)


Tool Schema Created:

                    Tool: add
                    Description: Add two numbers together.
                    Use when: you need to add two numbers
                    

                    Tool: subtract
                    Description: Subtract x from y (y - x).
                    Use when: you need to subtract one number from another
                    

                    Tool: multiply
                    Description: Multiply two numbers together.
                    Use when: you need to multiply two numbers
                    

                    Tool: exponentiate
                    Description: Raise x to the power of y (x^y).
                    Use when: you need to raise a number to a power
                    

                    Tool: final_answer
                    Description: Provide the final answer to the user.

    Args:
        answer: The final answer in natural language
        tools_used: List of tool names that were used
          

## Step 7: Building the Communication System


### Step 7a: Understanding Prompts

In [13]:
# A prompt is like instructions you give to the AI
# It needs to be very clear and specific

sample_prompt = """
You are a helpful assistant.
When I ask a math question, use the add tool.
Respond with: {"tool": "add", "parameters": {"x": 5, "y": 3}}
"""

print("A prompt is like giving instructions:")
print(sample_prompt)
print("\nThe clearer the instructions, the better the AI performs!")


A prompt is like giving instructions:

You are a helpful assistant.
When I ask a math question, use the add tool.
Respond with: {"tool": "add", "parameters": {"x": 5, "y": 3}}


The clearer the instructions, the better the AI performs!


### Step 7b: Creating Our Agent Prompt Template

In [14]:
from langchain_core.prompts import ChatPromptTemplate

# This is our "instruction manual" for the AI
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """🤖 You are a mathematical assistant with access to calculation tools.

AVAILABLE TOOLS:
{tool_schema}

 YOUR JOB:
1. Analyze the user's math question
2. Decide which tool to use
3. Respond with ONLY valid JSON in this format:

{{
    "thinking": "What calculation do I need to do?",
    "tool_name": "name_of_tool_to_use", 
    "parameters": {{"param1": value1, "param2": value2}}
}}

PREVIOUS WORK (if any):
{scratchpad}

IMPORTANT: 
- Always respond with valid JSON only
- Use final_answer when you have the complete answer
- Think step by step for complex problems"""),
    
    ("human", "{input}")
])

print("Agent prompt template created!")
print("This tells our AI exactly how to behave and respond")


Agent prompt template created!
This tells our AI exactly how to behave and respond


**Breaking down the prompt:**
- **System message**: The "rules" and instructions
- **Tool schema**: Description of available tools (filled in later)
- **Scratchpad**: Previous calculations (for multi-step problems)
- **JSON format**: Exact structure we expect back
- **Human message**: The user's actual question

## Step 8: Creating the JSON Response Parser

### Step 8a: Understanding the Parsing Challenge

In [15]:
# The AI might respond with extra text around the JSON
# We need to find and extract just the JSON part

examples = [
    '{"tool_name": "add", "parameters": {"x": 5, "y": 3}}',
    'I need to add these numbers: {"tool_name": "add", "parameters": {"x": 5, "y": 3}}',
    'Let me calculate:\n{"tool_name": "add", "parameters": {"x": 5, "y": 3}}\nThere you go!'
]

print("We need to handle different response formats:")
for i, example in enumerate(examples, 1):
    print(f"{i}. {example}")


We need to handle different response formats:
1. {"tool_name": "add", "parameters": {"x": 5, "y": 3}}
2. I need to add these numbers: {"tool_name": "add", "parameters": {"x": 5, "y": 3}}
3. Let me calculate:
{"tool_name": "add", "parameters": {"x": 5, "y": 3}}
There you go!


### Step 8b: Building a Simple JSON Finder

In [16]:
import re
import json

def find_json_in_text(text: str) -> str:
    """Find JSON object in text using pattern matching."""
    
    # Look for text that starts with { and ends with }
    json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
    
    matches = re.findall(json_pattern, text)
    
    if matches:
        return matches[0]  # Return the first JSON-like string found
    else:
        return text.strip()  # If no pattern found, return the whole text

# Test our JSON finder
test_text = 'The answer is: {"tool_name": "add", "parameters": {"x": 5, "y": 3}} - hope this helps!'
found_json = find_json_in_text(test_text)
print(f"Original: {test_text}")
print(f"Found JSON: {found_json}")

Original: The answer is: {"tool_name": "add", "parameters": {"x": 5, "y": 3}} - hope this helps!
Found JSON: {"tool_name": "add", "parameters": {"x": 5, "y": 3}}



### Step 8c: Building the Complete Parser

In [17]:
def parse_ai_response(response: str) -> Dict[str, Any]:
    """Parse the AI's response to extract tool call information."""
    
    print(f"Parsing response: {response[:100]}...")
    
    try:
        # Step 1: Find JSON in the response
        json_text = find_json_in_text(response)
        print(f"Extracted JSON: {json_text}")
        
        # Step 2: Parse the JSON
        parsed = json.loads(json_text)
        print(f"Successfully parsed JSON")
        
        # Step 3: Validate required fields
        required_fields = ["tool_name", "parameters"]
        for field in required_fields:
            if field not in parsed:
                print(f"Missing required field: {field}")
                parsed[field] = "final_answer" if field == "tool_name" else {}
        
        return parsed
        
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed: {e}")
        
        # Return a safe default
        return {
            "thinking": "Failed to parse response",
            "tool_name": "final_answer",
            "parameters": {
                "answer": "I'm sorry, I had trouble understanding the request.",
                "tools_used": []
            }
        }

# Test the parser
test_response = '{"thinking": "I need to add", "tool_name": "add", "parameters": {"x": 10, "y": 5}}'
parsed = parse_ai_response(test_response)
print(f"Final parsed result: {parsed}")


Parsing response: {"thinking": "I need to add", "tool_name": "add", "parameters": {"x": 10, "y": 5}}...
Extracted JSON: {"thinking": "I need to add", "tool_name": "add", "parameters": {"x": 10, "y": 5}}
Successfully parsed JSON
Final parsed result: {'thinking': 'I need to add', 'tool_name': 'add', 'parameters': {'x': 10, 'y': 5}}


**What this parser does:**
1. **Finds JSON**: Uses regex to locate JSON in messy text
2. **Parses safely**: Handles errors gracefully
3. **Validates**: Checks that required fields exist
4. **Provides fallback**: Returns safe default if parsing fails

## Step 9: Building the Tool Executor

### Step 9a: Understanding Tool Execution


In [18]:
# Once we know which tool to use, we need to actually run it
# This involves looking up the function and calling it with the right parameters

print("🔧 Tool execution process:")
print("1. Get tool name from parsed JSON")
print("2. Look up the actual function") 
print("3. Get parameters from parsed JSON")
print("4. Call the function with those parameters")
print("5. Handle any errors that might occur")

🔧 Tool execution process:
1. Get tool name from parsed JSON
2. Look up the actual function
3. Get parameters from parsed JSON
4. Call the function with those parameters
5. Handle any errors that might occur


### Step 9b: Creating a Safe Tool Executor

In [19]:
def execute_tool_safely(tool_name: str, parameters: Dict[str, Any]) -> Any:
    """Safely execute a tool with error handling."""
    
    print(f"Executing tool: {tool_name}")
    print(f"Parameters: {parameters}")
    
    # Step 1: Check if tool exists
    if tool_name not in name2tool:
        error_msg = f"Tool '{tool_name}' not found!"
        print(error_msg)
        return error_msg
    
    try:
        # Step 2: Get the actual function
        tool_function = name2tool[tool_name]
        print(f"Found tool function: {tool_function.__name__}")
        
        # Step 3: Execute the function
        result = tool_function(**parameters)
        print(f"Tool executed successfully")
        
        return result
        
    except TypeError as e:
        # This happens when parameters don't match the function signature
        error_msg = f"Parameter error for {tool_name}: {e}"
        print(error_msg)
        return error_msg
        
    except Exception as e:
        # This catches any other unexpected errors
        error_msg = f"Execution error for {tool_name}: {e}"
        print(error_msg)
        return error_msg

# Test the executor
test_result = execute_tool_safely("add", {"x": 10, "y": 5})
print(f"Test result: {test_result}")


Executing tool: add
Parameters: {'x': 10, 'y': 5}
Found tool function: add
Tool executed successfully
Test result: 15


**Why all this error handling?**
- The AI might request a tool that doesn't exist
- The AI might provide wrong parameter names
- The AI might provide wrong parameter types
- We want our agent to be robust and not crash

## Step 10: Building the Agent Brain (The Main Loop) 

### Step 10a: Understanding the Agent Loop

In [20]:
# Our agent needs to:
# 1. Take a user question
# 2. Think about what tool to use
# 3. Use that tool
# 4. Look at the result
# 5. Decide if it's done or needs more tools
# 6. Repeat until it has a final answer

print("🔄 The Agent Loop:")
print("┌─ User asks question")
print("│  ↓")
print("├─ Agent thinks about what to do") 
print("│  ↓")
print("├─ Agent uses a tool")
print("│  ↓")
print("├─ Agent sees result")
print("│  ↓")
print("├─ Done? → Give final answer")
print("│  ↓")
print("└─ Not done? → Think about next step (repeat)")


🔄 The Agent Loop:
┌─ User asks question
│  ↓
├─ Agent thinks about what to do
│  ↓
├─ Agent uses a tool
│  ↓
├─ Agent sees result
│  ↓
├─ Done? → Give final answer
│  ↓
└─ Not done? → Think about next step (repeat)


### Step 10b: Building the Simple Agent Class Structure

In [21]:
class SimpleOllamaAgent:
    """A simple agent that can use tools to solve math problems."""
    
    def __init__(self, max_steps: int = 5):
        """Initialize the agent.
        
        Args:
            max_steps: Maximum number of tool uses to prevent infinite loops
        """
        self.max_steps = max_steps
        self.llm = llm  # Our Ollama connection
        
        print(f"Agent initialized with max {max_steps} steps")
    
    def solve(self, question: str) -> str:
        """Solve a math problem step by step."""
        print(f"\nWorking on: {question}")
        return "I'm ready to solve problems!"

# Test creating an agent
agent = SimpleOllamaAgent(max_steps=3)
test_solve = agent.solve("What is 2 + 2?")
print(test_solve)


Agent initialized with max 3 steps

Working on: What is 2 + 2?
I'm ready to solve problems!


### Step 10c: Adding the Scratchpad System

In [22]:
def format_scratchpad(steps: List[str]) -> str:
    """Format previous steps for display to the AI."""
    
    if not steps:
        return "No previous calculations."
    
    formatted_steps = []
    for i, step in enumerate(steps, 1):
        formatted_steps.append(f"Step {i}: {step}")
    
    return "\n".join(formatted_steps)

# Test the scratchpad formatter
test_steps = [
    "Used add(5, 3) = 8",
    "Used multiply(8, 2) = 16"
]

formatted = format_scratchpad(test_steps)
print("Formatted scratchpad:")
print(formatted)


Formatted scratchpad:
Step 1: Used add(5, 3) = 8
Step 2: Used multiply(8, 2) = 16


**What's a scratchpad?**
- It's like showing your work in math class
- Keeps track of what the agent has already calculated
- Helps the agent build on previous results
- Makes the thinking process visible




### Step 10d: Implementing the Main Solve Method

In [23]:

class SimpleOllamaAgent:
    """A simple agent that can use tools to solve math problems."""
    
    def __init__(self, max_steps: int = 5):
        self.max_steps = max_steps
        self.llm = llm
        self.scratchpad = []  # Keep track of what we've done
        
    def solve(self, question: str) -> str:
        """Solve a math problem step by step."""
        
        print(f"\n Working on: {question}")
        self.scratchpad = []  # Start fresh for each question
        
        for step_number in range(1, self.max_steps + 1):
            print(f"\n--- Step {step_number} ---")
            
            # Step 1: Create the prompt with current context
            formatted_scratchpad = format_scratchpad(self.scratchpad)
            
            full_prompt = agent_prompt.format(
                tool_schema=tool_schema,
                scratchpad=formatted_scratchpad,
                input=question
            )
            
            print("Asking AI what to do next...")
            
            # Step 2: Get AI's response
            ai_response = self.llm.invoke(full_prompt)
            print(f"AI says: {ai_response}")
            
            # Step 3: Parse the response
            parsed = parse_ai_response(ai_response)
            
            tool_name = parsed["tool_name"]
            parameters = parsed["parameters"]
            thinking = parsed.get("thinking", "No reasoning provided")
            
            print(f"AI is thinking: {thinking}")
            print(f"Wants to use: {tool_name}({parameters})")
            
            # Step 4: Execute the tool
            result = execute_tool_safely(tool_name, parameters)
            
            # Normalize result summary for the scratchpad
            step_summary = f"Used {tool_name}({parameters}) → {result}"
            self.scratchpad.append(step_summary)
            
            # If the tool returned the special final_answer structure, finish
            if tool_name == "final_answer":
                # result is expected to be a dict like {"answer": "...", "tools_used": [...]}
                if isinstance(result, dict) and "answer" in result:
                    print(f"Done! Final answer: {result['answer']}")
                    return result["answer"]
                else:
                    # Unexpected shape; return a safe message
                    return "Agent returned final_answer but result format was unexpected."
            
            # Automatic finalize: if a math tool returned a numeric result, produce the final answer
            # This avoids asking the model to call final_answer for simple single-step math
            if isinstance(result, (int, float)):
                answer_text = str(result)
                tools_used = [tool_name]
                # Call the final_answer tool directly to keep consistent output format
                final_result = name2tool["final_answer"](answer=answer_text, tools_used=tools_used)
                if isinstance(final_result, dict) and "answer" in final_result:
                    print(f"Auto-finalized. Final answer: {final_result['answer']}")
                    return final_result["answer"]
                else:
                    return answer_text
        
        # If we used all steps without finishing
        return "I couldn't solve this within the allowed steps. Please try a simpler question."


# Create and test our agent
agent = SimpleOllamaAgent(max_steps=5)


In [24]:

import ast
import operator as _op

# safe eval helper for simple numeric expressions
_allowed_operators = {
    ast.Add: _op.add,
    ast.Sub: _op.sub,
    ast.Mult: _op.mul,
    ast.Div: _op.truediv,
    ast.Pow: _op.pow,
    ast.Mod: _op.mod,
    ast.USub: _op.neg,
    ast.UAdd: _op.pos,
}

def _eval_node(node):
    if isinstance(node, ast.Constant):  # Python 3.8+: ast.Num merged into Constant
        if isinstance(node.value, (int, float)):
            return node.value
        raise ValueError("Only numeric constants allowed")
    if isinstance(node, ast.Num):  # fallback for older ast
        return node.n
    if isinstance(node, ast.BinOp):
        left = _eval_node(node.left)
        right = _eval_node(node.right)
        op_type = type(node.op)
        if op_type in _allowed_operators:
            return _allowed_operators[op_type](left, right)
        raise ValueError(f"Operator {op_type} not allowed")
    if isinstance(node, ast.UnaryOp):
        operand = _eval_node(node.operand)
        op_type = type(node.op)
        if op_type in _allowed_operators:
            return _allowed_operators[op_type](operand)
        raise ValueError(f"Unary operator {op_type} not allowed")
    raise ValueError(f"Unsupported AST node: {type(node)}")

def safe_eval_expr(expr: str) -> float:
    """Safely evaluate a simple arithmetic expression and return numeric result."""
    expr = expr.strip()
    if expr == "":
        raise ValueError("Empty expression")
    # allow parentheses, numbers, and basic operators
    try:
        node = ast.parse(expr, mode="eval").body
        return _eval_node(node)
    except Exception as e:
        raise ValueError(f"Unsafe or invalid expression: {expr} ({e})")

def _coerce_value(val):
    """Coerce individual parameter value to numeric if it looks like an expression or numeric string."""
    # If already numeric, return as-is
    if isinstance(val, (int, float)):
        return val
    # If it's a boolean or None, keep as-is
    if isinstance(val, (bool, type(None))):
        return val
    # If it's a dict or list, coerce recursively
    if isinstance(val, dict):
        return {k: _coerce_value(v) for k, v in val.items()}
    if isinstance(val, list):
        return [_coerce_value(v) for v in val]
    # If string, try to convert to int/float first, then try safe eval for expressions like "(4 + 6)"
    if isinstance(val, str):
        s = val.strip()
        # remove surrounding quotes if present
        if (s.startswith("'") and s.endswith("'")) or (s.startswith('"') and s.endswith('"')):
            s = s[1:-1].strip()
        # try int
        try:
            return int(s)
        except Exception:
            pass
        # try float
        try:
            return float(s)
        except Exception:
            pass
        # try safe arithmetic eval if it contains digits and arithmetic chars
        if any(ch.isdigit() for ch in s) and any(ch in "+-*/%^() " for ch in s):
            # replace ^ with ** if user used caret
            s2 = s.replace("^", "**")
            try:
                return safe_eval_expr(s2)
            except Exception:
                pass
    # fallback: return original
    return val

def execute_tool_safely(tool_name: str, parameters: Dict[str, Any]) -> Any:
    """Safely execute a tool with error handling and coerce parameter types."""
    
    print(f"Executing tool: {tool_name}")
    print(f"Parameters: {parameters}")
    
    # Step 1: Check if tool exists
    if tool_name not in name2tool:
        error_msg = f"Tool '{tool_name}' not found!"
        print(error_msg)
        return error_msg
    
    try:
        # Step 2: Get the actual function
        tool_function = name2tool[tool_name]
        print(f"Found tool function: {tool_function.__name__}")
        
        # Coerce parameter values to numeric types where appropriate
        coerced_params = {}
        if isinstance(parameters, dict):
            for k, v in parameters.items():
                new_v = _coerce_value(v)
                if new_v != v:
                    print(f"Coerced parameter '{k}': {v} -> {new_v}")
                coerced_params[k] = new_v
        else:
            # If parameters is not a dict (unexpected), attempt to coerce whole object
            coerced_params = _coerce_value(parameters)
        
        # Step 3: Execute the function
        # If tool expects kwargs and we have a dict, pass as kwargs
        if isinstance(coerced_params, dict):
            result = tool_function(**coerced_params)
        else:
            # Otherwise attempt single-argument call
            result = tool_function(coerced_params)
        print(f"Tool executed successfully")
        
        return result
        
    except TypeError as e:
        # This happens when parameters don't match the function signature
        error_msg = f"Parameter error for {tool_name}: {e}"
        print(error_msg)
        return error_msg
        
    except Exception as e:
        # This catches any other unexpected errors
        error_msg = f"Execution error for {tool_name}: {e}"
        print(error_msg)
        return error_msg
# ...existing code...

## Step 11: Testing Our Agent Step by Step 

### Step 11a: Simple Addition Test

In [25]:
print("TEST 1: Simple Addition")
print("=" * 40)

result1 = agent.solve("What is 7 + 3?")
print(f"\nFinal Result: {result1}")


TEST 1: Simple Addition

 Working on: What is 7 + 3?

--- Step 1 ---
Asking AI what to do next...
AI says: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "add",
    "parameters": {"x": 7, "y": 3}
}
Parsing response: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "add",
    "parameters": {"x...
Extracted JSON: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "add",
    "parameters": {"x": 7, "y": 3}
}
Successfully parsed JSON
AI is thinking: What calculation do I need to do?
Wants to use: add({'x': 7, 'y': 3})
Executing tool: add
Parameters: {'x': 7, 'y': 3}
Found tool function: add
Tool executed successfully
Final Answer: 10
Tools Used: ['add']
Auto-finalized. Final answer: 10

Final Result: 10


### Step 11b: Two-Step Calculation Test

In [26]:
print("\nTEST 2: Two-Step Calculation")  
print("=" * 40)

result2 = agent.solve("What is (4 + 6) multiplied by 2?")
print(f"\nFinal Result: {result2}")



TEST 2: Two-Step Calculation

 Working on: What is (4 + 6) multiplied by 2?

--- Step 1 ---
Asking AI what to do next...
AI says: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "multiply",
    "parameters": {"x": "(4 + 6)", "y": "2"}
}
Parsing response: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "multiply",
    "parameters"...
Extracted JSON: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "multiply",
    "parameters": {"x": "(4 + 6)", "y": "2"}
}
Successfully parsed JSON
AI is thinking: What calculation do I need to do?
Wants to use: multiply({'x': '(4 + 6)', 'y': '2'})
Executing tool: multiply
Parameters: {'x': '(4 + 6)', 'y': '2'}
Found tool function: multiply
Coerced parameter 'x': (4 + 6) -> 10
Coerced parameter 'y': 2 -> 2
 Multiplying 10 × 2 = 20
Tool executed successfully
Final Answer: 20
Tools Used: ['multiply']
Auto-finalized. Final answer: 20

Final Result: 20


C:\Users\USERAS\AppData\Local\Temp\ipykernel_9500\3882710328.py:21: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # fallback for older ast


### Step 11c: Power/Exponent Test

In [27]:
print("\TEST 3: Exponentiation")
print("=" * 40)

result3 = agent.solve("What is 3 to the power of 4?")
print(f"\nFinal Result: {result3}")

\TEST 3: Exponentiation

 Working on: What is 3 to the power of 4?

--- Step 1 ---
Asking AI what to do next...


<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\USERAS\AppData\Local\Temp\ipykernel_9500\3121182763.py:1: SyntaxWarning: invalid escape sequence '\T'
  print("\TEST 3: Exponentiation")


AI says: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "exponentiate",
    "parameters": {"x": 3, "y": 4}
}
Parsing response: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "exponentiate",
    "paramet...
Extracted JSON: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "exponentiate",
    "parameters": {"x": 3, "y": 4}
}
Successfully parsed JSON
AI is thinking: What calculation do I need to do?
Wants to use: exponentiate({'x': 3, 'y': 4})
Executing tool: exponentiate
Parameters: {'x': 3, 'y': 4}
Found tool function: exponentiate
 Calculating 3^4 = 81
Tool executed successfully
Final Answer: 81
Tools Used: ['exponentiate']
Auto-finalized. Final answer: 81

Final Result: 81


**What to look for in the output:**
- Each step should be clearly numbered
- You should see the AI's thinking process
- Tool executions should show parameters and results
- The scratchpad should build up over multiple steps

## Step 12: Adding Memory for Conversations

### Step 12a: Understanding Conversation Memory


In [32]:
class MemoryOllamaAgent(SimpleOllamaAgent):
    """An agent with conversation memory."""
    
    def __init__(self, max_steps: int = 5):
        super().__init__(max_steps)
        self.conversation_history = []  # Remember past conversations
        
    def solve(self, question: str) -> str:
        """Solve a problem and remember the conversation.
        
        Includes recent conversation context in the prompt and saves final answers
        (whether produced by final_answer tool or auto-finalized numeric results).
        """
        print(f"\nWorking on: {question}")
        
        # Show conversation history if it exists
        if self.conversation_history:
            print("Conversation history:")
            for i, (q, a) in enumerate(self.conversation_history[-3:], 1):  # Show last 3
                print(f"  {i}. Q: {q}")
                print(f"     A: {a}")
        
        # Solve the problem (similar loop as SimpleOllamaAgent but include context)
        self.scratchpad = []
        
        for step_number in range(1, self.max_steps + 1):
            print(f"\n--- Step {step_number} ---")
            
            formatted_scratchpad = format_scratchpad(self.scratchpad)
            context = self._build_context()
            
            full_prompt = agent_prompt.format(
                tool_schema=tool_schema,
                scratchpad=formatted_scratchpad + "\n\n" + context,
                input=question
            )
            
            print("Asking AI what to do next...")
            ai_response = self.llm.invoke(full_prompt)
            print(f"AI says: {ai_response}")
            
            parsed = parse_ai_response(ai_response)
            tool_name = parsed["tool_name"]
            parameters = parsed["parameters"]
            thinking = parsed.get("thinking", "No reasoning provided")
            
            print(f" AI is thinking: {thinking}")
            print(f" Wants to use: {tool_name}({parameters})")
            
            result = execute_tool_safely(tool_name, parameters)
            step_summary = f"Used {tool_name}({parameters}) → {result}"
            self.scratchpad.append(step_summary)
            
            # If the tool returned the special final_answer structure, finish and save
            if tool_name == "final_answer":
                if isinstance(result, dict) and "answer" in result:
                    final_answer = result["answer"]
                else:
                    final_answer = "Agent returned final_answer but result format was unexpected."
                # Save to conversation history
                self.conversation_history.append((question, final_answer))
                print(f" Done! Final answer: {final_answer}")
                return final_answer
            
            # Auto-finalize numeric results (same behavior as SimpleOllamaAgent)
            if isinstance(result, (int, float)):
                answer_text = str(result)
                tools_used = [tool_name]
                final_result = name2tool["final_answer"](answer=answer_text, tools_used=tools_used)
                if isinstance(final_result, dict) and "answer" in final_result:
                    final_answer = final_result["answer"]
                else:
                    final_answer = answer_text
                # Save to conversation history
                self.conversation_history.append((question, final_answer))
                print(f"Auto-finalized. Final answer: {final_answer}")
                return final_answer
        
        return "I couldn't solve this within the allowed steps."
    
    def _build_context(self) -> str:
        """Build context from recent conversations."""
        if not self.conversation_history:
            return "No previous conversation."
        
        context_parts = [" Recent conversation:"]
        for q, a in self.conversation_history[-2:]:  # Last 2 conversations
            context_parts.append(f"Q: {q}")
            context_parts.append(f"A: {a}")
        
        return "\n".join(context_parts)

# Create the new agent with memory
memory_agent = MemoryOllamaAgent(max_steps=5)


In [33]:
memory_agent = MemoryOllamaAgent(max_steps=5)

### Step 12c: Testing Memory

In [34]:
print("TEST 4: Memory Test")
print("=" * 40)

# First question
result1 = memory_agent.solve("What is 5 + 10?")

print("\n" + "-" * 40)

# Second question that might reference the first
result2 = memory_agent.solve("Now multiply that result by 2")

print(f"\nConversation History Length: {len(memory_agent.conversation_history)}")


TEST 4: Memory Test

Working on: What is 5 + 10?

--- Step 1 ---
Asking AI what to do next...
AI says: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "add",
    "parameters": {"x": 5, "y": 10}
}
Parsing response: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "add",
    "parameters": {"x...
Extracted JSON: {
    "thinking": "What calculation do I need to do?",
    "tool_name": "add",
    "parameters": {"x": 5, "y": 10}
}
Successfully parsed JSON
 AI is thinking: What calculation do I need to do?
 Wants to use: add({'x': 5, 'y': 10})
Executing tool: add
Parameters: {'x': 5, 'y': 10}
Found tool function: add
Tool executed successfully
Final Answer: 15
Tools Used: ['add']
Auto-finalized. Final answer: 15

----------------------------------------

Working on: Now multiply that result by 2
Conversation history:
  1. Q: What is 5 + 10?
     A: 15

--- Step 1 ---
Asking AI what to do next...
AI says: {
    "thinking": "What calculation do I ne